In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib
import sys

In [2]:
sys.path.insert(0, '..')

In [3]:
from encoder_decoder_classes import EncoderDecoderConvLSTM

Below, dataset is the original dataset, with non-normalized features and containing nan values. dataset_norm has the features normalized and nan values replaced by 0.

In [4]:
dataset = torch.tensor(np.load('../../../numpy_dataset/dataset.npy'))

In [5]:
dataset_norm = torch.tensor(np.load('../../../numpy_dataset/dataset_normalized.npy'))

The input X consists of the first 11 days of data (the current day preceded by 10 days of lag). The target y consists of the rest of the data (30 days of lead).

In [6]:
X = dataset_norm[:,:11,:,:,:]

In [7]:
y = dataset_norm[:,11:,:,:,:]

y_un is pulled from dataset and thus contains nan values.

In [8]:
y_un = dataset[:,11:,:,:,:]

In [9]:
future_steps = 30

In [10]:
model = EncoderDecoderConvLSTM()

In [11]:
model.load_state_dict(torch.load('../best_model_params.pt', map_location=torch.device('cpu')))

In [12]:
model.eval()

In [13]:
test_len = int(len(X) * 0.2)

In [14]:
the_range = list(range(len(X) - test_len, len(X)))

We'll take 20 random samples from the test set and graph the predictions of the model on those 20 random samples.

In [15]:
np.random.seed(42)

In [16]:
sample_indices = np.random.choice(the_range, size=20, replace=False)

In [17]:
sample_indices

In [18]:
sample = X[sample_indices,:,:,:,:]

In [19]:
y_pred = model(sample).squeeze(2).detach().numpy()

In [20]:
y_pred.shape

In [21]:
y_true = y[sample_indices,:,1,:,:].numpy()

In [22]:
y_true.shape

Below, we mask y_pred and y_true using the nan values in y_un in order to display the predictions sensibly in a pdf.

In [23]:
for_mask = y_un[sample_indices,:,1,:,:]

In [24]:
y_pred = np.ma.masked_where(for_mask.isnan(), y_pred)

In [25]:
y_true = np.ma.masked_where(for_mask.isnan(), y_true)

In [26]:
cmap = plt.get_cmap('Blues')

In [27]:
cmap.set_bad(color='green')

Here we graph the predictions of the model on each of the 20 random test samples. 

In [28]:
for i, ind in enumerate(sample_indices):
    y_pred_ = y_pred[i]
    y_true_ = y_true[i]

    fig, axes = plt.subplots(30,2)

    fig.set_figheight(45)
    
    fig.suptitle(f'Predicted versus actual oxygen for datapoint {ind}', fontsize=10)
    
    for j in range(30):
        axs = axes[j]
        axs[0].get_xaxis().set_ticks([])
        axs[0].get_yaxis().set_ticks([])
        axs[1].get_xaxis().set_ticks([])
        axs[1].get_yaxis().set_ticks([])
        t = axs[0].imshow(y_true_[j], cmap=cmap, vmin=-2, vmax=2, origin='lower')
        p = axs[1].imshow(y_pred_[j], cmap=cmap, vmin=-2, vmax=2, origin='lower')
        fig.colorbar(t, ax=axs[0], location='right')
        fig.colorbar(p, ax=axs[1], location='right')
    
    plt.savefig(f'graph_point_{ind}.pdf', format='pdf')
    plt.show()